# Project Objective
Use bioinformatics tools to identify potential drugs for the treatment of anthrax.

# Acquire

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='ticks')

from numpy.random import seed
seed(1)

from numpy.random import randn

from scipy.stats import mannwhitneyu

#pip install chembl_webresource_client that gets data from ChEMBL server
from chembl_webresource_client.new_client import new_client

#conda install -c conda-forge rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

# Installs the python library version of wget, be careful because there are different versions
#pip install wget
import wget

# This is used to unzip files
from zipfile import ZipFile

# This library calculates chemical fingerprints from SMILES chemical notation
#pip install padelpy
from padelpy import padeldescriptor

# This library can gather files of a desired type together into a list
import glob

from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.feature_selection import VarianceThreshold

import os

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

RDKit WARNING: [09:15:34] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
def acquire_data(disease):
    '''
    This function gets disease data from local csv, or otherwise from ChEMBL database.
    '''
    
    if os.path.isfile('disease_chembl_data.csv'):
        df = pd.read_csv('disease_chembl_data.csv', index_col = 0)
    
    else:
        # Create and use new_client object to access ChEMBL database
        target = new_client.target
        target_query = target.search(disease)
        df = pd.DataFrame.from_dict(target_query)
        df.to_csv('disease_chembl_data.csv')
    
    return df

df = acquire_data('coronavirus')
df.head()

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Coronavirus,Coronavirus,17.0,False,CHEMBL613732,[],ORGANISM,11119
1,[],SARS coronavirus,SARS coronavirus,15.0,False,CHEMBL612575,[],ORGANISM,227859
2,[],Feline coronavirus,Feline coronavirus,15.0,False,CHEMBL612744,[],ORGANISM,12663
3,[],Human coronavirus 229E,Human coronavirus 229E,13.0,False,CHEMBL613837,[],ORGANISM,11137
4,"[{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...",SARS coronavirus,SARS coronavirus 3C-like proteinase,10.0,False,CHEMBL3927,"[{'accession': 'P0C6U8', 'component_descriptio...",SINGLE PROTEIN,227859


In [3]:
# This is the list of targets within our disease
# We need to pick one target to proceed
# This notebook can be run again with different targets chosen
df.target_chembl_id

0     CHEMBL613732
1     CHEMBL612575
2     CHEMBL612744
3     CHEMBL613837
4       CHEMBL3927
5    CHEMBL4296578
6       CHEMBL5118
7    CHEMBL4523582
Name: target_chembl_id, dtype: object

In [5]:
# Focusing on 'Replicase polyprotein 1ab' on index row 4
selected_target = df.target_chembl_id[4]

def get_bioactivity_data(selected_target):
    '''
    This function get the bioactivity of compounds against the disease target.
    '''
    
    if os.path.isfile('bioactivity_data.csv'):
        df = pd.read_csv('bioactivity_data.csv', index_col = 0)
    
    else:
        # Get bioactivity data for our target coronavirus protein
        # The standard_type='IC50' filters for bioactivity tests using the IC50 standard of measuring
        # 'activity' will be a list of dictionaries
        activity = new_client.activity.filter(target_chembl_id = selected_target).filter(standard_type='IC50')
        
        # Turn the 'activity' list of dictionaries into a pandas dataframe
        df = pd.DataFrame.from_dict(activity)
        # standard_value column represents potency
        # A smaller number means a smaller dose is needed to exhibit and effect
        # Lower value means more potent, higher value means less potent
        
        # Save a local copy of our bioactivity data
        df.to_csv('bioactivity_data.csv', index=False)
    
    return df

bioactivity_df = get_bioactivity_data(selected_target)
bioactivity_df

,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
activity_comment,,,,,,,,,,,,,,,,,,,,,
NaN,1480935,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,NaN,NaN,BAO_0000190,BAO_0000357,single protein format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,7.20
NaN,1480936,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,NaN,NaN,BAO_0000190,BAO_0000357,single protein format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,9.40
NaN,1481061,[],CHEMBL830868,In vitro inhibitory concentration against SARS...,B,NaN,NaN,BAO_0000190,BAO_0000357,single protein format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,13.50
NaN,1481065,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,NaN,NaN,BAO_0000190,BAO_0000357,single protein format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,13.11
NaN,1481066,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,NaN,NaN,BAO_0000190,BAO_0000357,single protein format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,12041507,[],CHEMBL2150313,Inhibition of SARS-CoV PLpro expressed in Esch...,B,NaN,NaN,BAO_0000190,BAO_0000019,assay format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,10.60
NaN,12041508,[],CHEMBL2150313,Inhibition of SARS-CoV PLpro expressed in Esch...,B,NaN,NaN,BAO_0000190,BAO_0000019,assay format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,10.10
NaN,12041509,[],CHEMBL2150313,Inhibition of SARS-CoV PLpro expressed in Esch...,B,NaN,NaN,BAO_0000190,BAO_0000019,assay format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,11.50


In [ ]:
df.info()

In [ ]:
# look for null values in the standard_value feature
df.standard_value.isna().sum()

In [ ]:
# look for null values in the canonical_smiles feature
df.canonical_smiles.isna().sum()

# Prepare

In [ ]:
def classify_bioactivity():
    '''
    Divide the compounds into classes of potency: inactive, active, and intermediate.
    '''
    
    bioactivity_class = []
    for i in df.standard_value:
        if float(i) >= 10000:
            bioactivity_class.append('inactive')
        elif float(i) <= 1000:
            bioactivity_class.append('active')
        else:
            bioactivity_class.append('intermediate')
    
    return bioactivity_class

bioactivity_class = classify_bioactivity()
bioactivity_class

In [ ]:
# This is a list of the chemical compounds tested against our disease target
df.molecule_chembl_id

In [ ]:
# bioactivity_class has the same length as df.molecule_chembl_id
len(bioactivity_class)

In [ ]:
## Drop unneeded features from the dataframe
#selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
#df = df[selection]
#
## Add column for bioactivity_class
#df = pd.concat([df, pd.Series(bioactivity_class)], axis=1 )
#df = df.rename(columns={0:'bioactivity_class'})
#
#df

def get_lipinski_parameters(df):
    '''
    Calculate the lipinski parameters of the compounds.
    '''
    
    # Drop unneeded features from the dataframe
    selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
    df = df[selection]
    
    # Classify the bioactivity of the compounds
    bioactivity_class = classify_bioactivity()
    
    # Add column for bioactivity_class
    df = pd.concat([df, pd.Series(bioactivity_class)], axis=1 )
    df = df.rename(columns={0:'bioactivity_class'})
    
    # Save a local copy of the pre-processed bioactivity data
    df.to_csv('bioactivity_preprocessed_data.csv', index=False)
    
    return df

df = get_lipinski_parameters(df)
df

In [ ]:
# Check how the bioactivity classes are distributed for the chenmical compounds
df.bioactivity_class.value_counts()

# Explore

In [ ]:
# calculate lipinski descriptors (chemical parameters that predict druglikeness of chemical compunds)
# SMILES is a chemical notation that describes a compounds chemical structure
# This function gives the four parameters described by Lipinski's rule of five
def lipinski(smiles, verbose=False):
    moldata = []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)
        
    baseData = np.arange(1,1)
    i = 0
    for mol in moldata:
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
        
        row = np.array([desc_MolWt, desc_MolLogP, desc_NumHDonors, desc_NumHAcceptors])
        
        if(i==0):
            baseData = row
        else:
            baseData = np.vstack([baseData, row])
        
        i = i + 1
        
    columnNames = ['MW', 'LogP', 'NumHDonors', 'NumHAcceptors']
    descriptors = pd.DataFrame(data = baseData, columns = columnNames)
    
    return descriptors

In [ ]:
# This dataframe holds the four parameters described by Lipinski's rule of five
# These chemical parameters can be used to estimate the druglikeness of a compound
# This df has the same number of rows as df
lipinskidf = lipinski(df.canonical_smiles)
lipinski_df

In [ ]:
df = pd.concat([df, lipinski_df], axis=1) # combine the df and lipinski_df
df.standard_value = df.standard_value.astype('float64') # Convert standard_value to numeric type for calculations
df

In [ ]:
# Columns that will be used for calculations have numeric type
df.info()

In [ ]:
# If the standard_value >100,000,000 then it will give negative values after taking the negative logarithm
# Thus, we need to check if values are >100,000,000
# Our max standard_value is too large
df.standard_value.describe()

In [ ]:
# Our max standard_value is too large, therefore we will cap it at 100,000,000
# This prevents getting nevative values from the negative log, which will make interpretation more difficult

# This function caps standard_value at 100,000,000
def norm_value(input):
    norm = []
    
    for i in input['standard_value']:
        if i > 100000000:
            i = 100000000
        norm.append(i)
    
    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
    
    return x  

In [ ]:
# Cap the standard_value
df = norm_value(df)
df

In [ ]:
# After capping, our max standard_value is <= 100,000,000
df.standard_value_norm.describe()

In [ ]:
# Convert IC50 to pIC50
# This is done because IC50 in general has an uneven distribution
# Taking the negative log makes the distribution of IC50 more even

def pIC50(input):
    pIC50 = []
    
    for i in input['standard_value_norm']:
        molar = i * (10**-9) # converts nanomolar to molar
        pIC50.append(-np.log10(molar))
        
    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
    
    return x

In [ ]:
# Convert the IC50 column to pIC50
df = pIC50(df)
df

In [ ]:
df.pIC50.describe()

In [ ]:
# Remove compounds with an intermediate bioactivity class
# We only lose 14 rows after dropping intermediate compounds
df = df[df.bioactivity_class != 'intermediate']
df

In [ ]:
# Frequency plot of bioactivity classes
plt.figure(figsize=(5, 5))
sns.countplot(x='bioactivity_class', data=df, edgecolor='black')
plt.xlabel('Bioactivity Class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')
#plt.savefig('plot_bioactivity_class.pdf')
plt.show()

In [ ]:
# The two bioactivity classes are spanning similar chemical spaces as shown in the plot below

# Scatter plot of MW vs LogP
plt.figure(figsize=(5, 5))
sns.scatterplot(x='MW', y='LogP', data=df, hue='bioactivity_class', size='pIC50', edgecolor='black', alpha=0.7)
plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0) # this creates a legend and places it outside of plot
#plt.savefig('plot_MW_vs_LogP.pdf')
plt.show()

In [ ]:
# This verifies that we separated compounds into active and inactive groups

plt.figure(figsize=(5, 5))
sns.boxplot(x='bioactivity_class', y='pIC50', data=df)
plt.xlabel('Bioactivity Class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50', fontsize=14, fontweight='bold')
#plt.savefig('plot_pIC50.pdf')
plt.show()

In [ ]:
def mannwhitney(descriptor, verbose=False):
    # 'descriptor' is the column in the df that we are testing
    # https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
    
    # Set the active and inactive
    selection = [descriptor, 'bioactivity_class']
    df_stat = df[selection]
    
    active = df_stat[df_stat.bioactivity_class == 'active']
    active = active[descriptor]
    
    inactive = df_stat[df_stat.bioactivity_class == 'inactive']
    inactive = inactive[descriptor]
    
    # Compare the active and inactive groups
    stat, p = mannwhitneyu(active, inactive)
    
    # Interpret the results
    alpha = 0.05
    if p > alpha:
        interpretation = 'Same distribution (fail to reject H0)'
    else:
        interpretation = 'Different distribution (reject H0)'
    
    results = pd.DataFrame({'Descriptor':descriptor, 
                            'Statistics':stat, 
                            'p':p, 
                            'alpha':alpha, 
                            'Interpretation':interpretation}, index=[0])
    
    # Save a local copy of the statistical results
    #filename = 'mannwhitneyu_' + descriptor + '.csv'
    #results.to_csv(filename)
    
    return results

In [ ]:
# Compare active and inactive bioactivity classes for differences in pIC50
# There is a statistically significant difference between active and inactive for pIC50
mannwhitney('pIC50')

In [ ]:
plt.figure(figsize=(5, 5))
sns.boxplot(x='bioactivity_class', y='MW', data=df)
plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')
#plt.savefig('plot_MW.pdf')
plt.show()

In [ ]:
plt.figure(figsize=(5, 5))
sns.boxplot(x='bioactivity_class', y='LogP', data=df)
plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
#plt.savefige('plot_LogP.pdf')
plt.show()

In [ ]:
# There is not a statistically significant difference between active and inactive for LogP
mannwhitney('LogP')

In [ ]:
plt.figure(figsize=(5, 5))
sns.boxplot(x='bioactivity_class', y='NumHDonors', data=df)
plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Number of H Donors', fontsize=14, fontweight='bold')
#plt.savefige('plot_NumHDonors.pdf')
plt.show()

In [ ]:
# There is a statistically significant difference between active and inactive for the number of hydrogen donors
mannwhitney('NumHDonors')

In [ ]:
plt.figure(figsize=(5, 5))
sns.boxplot(x='bioactivity_class', y='NumHAcceptors', data=df)
plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Number of H Acceptors', fontsize=14, fontweight='bold')
#plt.savefige('plot_NumHAcceptors.pdf')
plt.show()

In [ ]:
# There is a statistically significant difference between active and inactive for the number of hydrogen acceptors
mannwhitney('NumHAcceptors')

Boxplots <br>
pIC50 values were statistically significant, which is expected because we classified active vs inactive by IC50 <br> <br>
Lipinski descriptors <br>
of the four Lipinski descriptors, only LogP show no difference between active and inactive compounds. The other three Lipinski descriptors showed statistically significant differences between active and inactive.


In [ ]:
# Download xml files that tell padelpy what kind of fingerprints to make
# Only run this once, because subsequent times it we add a space and a number to the name of the zip file
# The space in the name causes errors in the next cell
url = 'https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip'
filename = wget.download(url)
filename

In [ ]:
# Unzip fingerprints_xml.zip
# Create a zip object
with ZipFile(filename, 'r') as zipObj:
    zipObj.extractall() # Extract all contents of zip file into the current directory

In [ ]:
# Create a sorted list of xml files
xml_files = glob.glob('*.xml')
xml_files.sort()
xml_files

In [ ]:
# Create a list of file names
FP_list = ['AtomPairs2DCount', 'AtomPairs2D', 'EState', 'CDKextended', 'CDK', 'CDKgraphonly', 
           'KlekotaRothCount', 'KlekotaRoth', 'MACCS', 'PubChem', 'SubstructureCount', 'Substructure']

In [ ]:
# Create data dictionary
fp = dict(zip(FP_list, xml_files))
fp

In [ ]:
fp_df = pd.concat([df.canonical_smiles, df.molecule_chembl_id], axis=1)
fp_df

In [ ]:
#selection = pd.concat([candidate_df.canonical_smiles, candidate_df.molecule_chembl_id], axis=1)
# Alternative method
selection = ['canonical_smiles','molecule_chembl_id']
fP_df_selection = df[selection]
fP_df_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)
fP_df_selection

In [ ]:
# Setting up the module to calculate the fingerprints using the Pubchem style
fingerprint = 'PubChem'

fingerprint_output_file = ''.join([fingerprint, '.csv']) #Substructure.csv

fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='molecule.smi', 
                d_file=fingerprint_output_file, # 'Substructure.csv'
                descriptortypes=fingerprint_descriptortypes, # descriptortypes='SubstructureFingerprint.xml'
                detectaromaticity=True, 
                standardizenitro=True, 
                standardizetautomers=True, 
                threads=2, 
                removesalt=True, 
                log=True, 
                fingerprints=True)

In [ ]:
# The padelpy function about printed out the results to csv, which we will now read
descriptors = pd.read_csv(fingerprint_output_file)
descriptors.head()

In [ ]:
# Drop 'Name' as a feature, so that the only columns are chemical fingerprints
data_x = descriptors.drop(columns='Name')
data_x

In [ ]:
# Data X and Y have the same number of rows
data_y = df['pIC50'].reset_index().drop(columns='index')
data_y

In [ ]:
# Combine to get a dataframe with predictors and target for machine learning models
data_xy = pd.concat([data_x ,data_y], axis=1)

# Create a local copy of csv, this can avoid the need to repeat computationally expensive steps
data_xy.to_csv('chemical_fingerprints.csv', index=False)

In [ ]:
# Remove low variance features from the dataframe
# Low variance features are unlikely to be valuable for predicting, but increase computational cost
selection = VarianceThreshold(threshold=(.8 * (1 - .8)))    
data_x = selection.fit_transform(data_x)

In [ ]:
# Number of columns reduced from 883 down to 197
data_x.shape

In [ ]:
# Split the data with 80/20 ratio
X_train, X_test, Y_train, Y_test = train_test_split(data_x, data_y, test_size=0.2)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
X_test.shape, Y_test.shape

In [ ]:
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, Y_train)
r2 = model.score(X_test, Y_test)
r2

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
sns.set(color_codes=True)
sns.set_style("white")

ax = sns.regplot(Y_test, Y_pred, scatter_kws={'alpha':0.4})
ax.set_xlabel('Experimental pIC50', fontsize='large', fontweight='bold')
ax.set_ylabel('Predicted pIC50', fontsize='large', fontweight='bold')
ax.set_xlim(0, 12)
ax.set_ylim(0, 12)
ax.figure.set_size_inches(5, 5)
plt.show